<a href="https://colab.research.google.com/github/Sk0108/Log-Classification-System/blob/main/LogClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pandas

In [2]:
import pandas as pd

from google.colab import drive
drive.mount('/content/drive')

df= pd.read_csv("/content/drive/MyDrive/synthetic_logs.csv")
print(df.head())
print(df.source.unique())
print(df.target_label.unique())

Mounted at /content/drive
             timestamp           source  \
0  2025-06-27 07:20:25        ModernCRM   
1      1/14/2025 23:07        ModernCRM   
2       1/17/2025 1:29  AnalyticsEngine   
3  2025-07-12 00:24:16         ModernHR   
4  2025-06-02 18:25:23    BillingSystem   

                                         log_message    target_label  \
0  nova.osapi_compute.wsgi.server [req-b9718cd8-f...     HTTP Status   
1     Email service experiencing issues with sending  Critical Error   
2          Unauthorized access to data was attempted  Security Alert   
3  nova.osapi_compute.wsgi.server [req-4895c258-b...     HTTP Status   
4  nova.osapi_compute.wsgi.server [req-ee8bc8ba-9...     HTTP Status   

  complexity  
0       bert  
1       bert  
2       bert  
3       bert  
4       bert  
['ModernCRM' 'AnalyticsEngine' 'ModernHR' 'BillingSystem' 'ThirdPartyAPI'
 'LegacyCRM']
['HTTP Status' 'Critical Error' 'Security Alert' 'Error'
 'System Notification' 'Resource Usage' 'User A

In [3]:
from sklearn.cluster import DBSCAN
from sentence_transformers import SentenceTransformer

In [4]:
model = SentenceTransformer('all-MiniLM-L6-v2')  # Lightweight embedding model
embeddings = model.encode(df['log_message'].tolist())

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [5]:
print(embeddings[:5])

[[-0.1029396   0.03354595 -0.02202608 ...  0.00457793 -0.04259716
   0.00322623]
 [ 0.00804573 -0.03573924  0.04938739 ...  0.01538321 -0.06230951
  -0.02774664]
 [-0.00908221  0.13003927 -0.05275569 ...  0.02014104 -0.05117096
  -0.02930295]
 [-0.09751045  0.04911302 -0.03977428 ...  0.02477501 -0.03546082
  -0.00018599]
 [-0.10468338  0.05926034 -0.02488501 ...  0.02502053 -0.03719296
  -0.02568914]]


In [6]:
clustering = DBSCAN(eps=0.2, min_samples=1, metric='cosine').fit(embeddings)
df['cluster'] = clustering.labels_

In [7]:
print(df.head())

             timestamp           source  \
0  2025-06-27 07:20:25        ModernCRM   
1      1/14/2025 23:07        ModernCRM   
2       1/17/2025 1:29  AnalyticsEngine   
3  2025-07-12 00:24:16         ModernHR   
4  2025-06-02 18:25:23    BillingSystem   

                                         log_message    target_label  \
0  nova.osapi_compute.wsgi.server [req-b9718cd8-f...     HTTP Status   
1     Email service experiencing issues with sending  Critical Error   
2          Unauthorized access to data was attempted  Security Alert   
3  nova.osapi_compute.wsgi.server [req-4895c258-b...     HTTP Status   
4  nova.osapi_compute.wsgi.server [req-ee8bc8ba-9...     HTTP Status   

  complexity  cluster  
0       bert        0  
1       bert        1  
2       bert        2  
3       bert        0  
4       bert        0  


In [8]:
# Group by cluster to inspect patterns
clusters = df.groupby('cluster')['log_message'].apply(list)
sorted_clusters = clusters.sort_values(key=lambda x: x.map(len), ascending=False)

In [9]:
print("Clustered Patterns:")
for cluster_id, messages in sorted_clusters.items():
    if len(messages) > 10:
        print(f"Cluster {cluster_id}:")
        for msg in messages[:5]:
            print(f"  {msg}")

Clustered Patterns:
Cluster 0:
  nova.osapi_compute.wsgi.server [req-b9718cd8-f65e-49cc-8349-6cf7122af137 113d3a99c3da401fbd62cc2caa5b96d2 54fadb412c4e40cdbaed9335e4c35a9e - - -] 10.11.10.1 "GET /v2/54fadb412c4e40cdbaed9335e4c35a9e/servers/detail HTTP/1.1" status: 200 len: 1893 time: 0.2675118
  nova.osapi_compute.wsgi.server [req-4895c258-b2f8-488f-a2a3-4fae63982e48 113d3a99c3da401fbd62cc2caa5b96d2 54fadb412c4e40cdbaed9335e4c35a9e - - -] 10.11.10.1 "GET /v2/54fadb412c4e40cdbaed9335e4c35a9e/servers/detail HTTP/1.1" HTTP status code -  200 len: 211 time: 0.0968180
  nova.osapi_compute.wsgi.server [req-ee8bc8ba-9265-4280-9215-dbe000a41209 113d3a99c3da401fbd62cc2caa5b96d2 54fadb412c4e40cdbaed9335e4c35a9e - - -] 10.11.10.1 "GET /v2/54fadb412c4e40cdbaed9335e4c35a9e/servers/detail HTTP/1.1" RCODE  200 len: 1874 time: 0.2280791
  nova.osapi_compute.wsgi.server [req-f0bffbc3-5ab0-4916-91c1-0a61dd7d4ec2 113d3a99c3da401fbd62cc2caa5b96d2 54fadb412c4e40cdbaed9335e4c35a9e - - -] 10.11.10.1 "GET /v2

## **Classification stage 1: RegEX**

In [10]:
import re
def classify_with_regex(log_message):
    regex_patterns = {
        r"User User\d+ logged (in|out).": "User Action",
        r"Backup (started|ended) at .*": "System Notification",
        r"Backup completed successfully.": "System Notification",
        r"System updated to version .*": "System Notification",
        r"File .* uploaded successfully by user .*": "System Notification",
        r"Disk cleanup completed successfully.": "System Notification",
        r"System reboot initiated by user .*": "System Notification",
        r"Account with ID .* created by .*": "User Action"
    }
    for pattern, label in regex_patterns.items():
        if re.search(pattern, log_message):
            return label
    return None

In [11]:
classify_with_regex("User User123 logged in.")

'User Action'

In [12]:
classify_with_regex("System reboot initiated by user User179.")

'System Notification'

In [13]:
classify_with_regex("Hey you, chill bro")

In [14]:
# Apply regex classification
df['regex_label'] = df['log_message'].apply(lambda x: classify_with_regex(x))
df[df['regex_label'].notnull()]

,timestamp,source,log_message,target_label,complexity,cluster,regex_label
7,10/11/2025 8:44,ModernHR,File data_6169.csv uploaded successfully by us...,System Notification,regex,4,System Notification
14,1/4/2025 1:43,ThirdPartyAPI,File data_3847.csv uploaded successfully by us...,System Notification,regex,4,System Notification
15,5/1/2025 9:41,ModernCRM,Backup completed successfully.,System Notification,regex,8,System Notification
18,2/22/2025 17:49,ModernCRM,Account with ID 5351 created by User634.,User Action,regex,9,User Action
27,9/24/2025 19:57,ThirdPartyAPI,User User685 logged out.,User Action,regex,11,User Action
...,...,...,...,...,...,...,...
2376,6/27/2025 8:47,ModernCRM,System updated to version 2.0.5.,System Notification,regex,21,System Notification
2381,9/5/2025 6:39,ThirdPartyAPI,Disk cleanup completed successfully.,System Notification,regex,32,System Notification
2394,4/3/2025 13:13,ModernHR,Disk cleanup completed successfully.,System Notification,regex,32,System Notification
2395,5/2/2025 14:29,ThirdPartyAPI,Backup ended at 2025-05-06 11:23:16.,System Notification,regex,13,System Notification


In [15]:
print(df[df['regex_label'].isnull()].head(5))

             timestamp           source  \
0  2025-06-27 07:20:25        ModernCRM   
1      1/14/2025 23:07        ModernCRM   
2       1/17/2025 1:29  AnalyticsEngine   
3  2025-07-12 00:24:16         ModernHR   
4  2025-06-02 18:25:23    BillingSystem   

                                         log_message    target_label  \
0  nova.osapi_compute.wsgi.server [req-b9718cd8-f...     HTTP Status   
1     Email service experiencing issues with sending  Critical Error   
2          Unauthorized access to data was attempted  Security Alert   
3  nova.osapi_compute.wsgi.server [req-4895c258-b...     HTTP Status   
4  nova.osapi_compute.wsgi.server [req-ee8bc8ba-9...     HTTP Status   

  complexity  cluster regex_label  
0       bert        0        None  
1       bert        1        None  
2       bert        2        None  
3       bert        0        None  
4       bert        0        None  


## Classification Stage 2: **Classification** Using embeddings

In [16]:
df_non_regex = df[df['regex_label'].isnull()].copy()
print(df_non_regex.shape)

(1910, 7)


In [17]:
df_legacy = df_non_regex[df_non_regex.source=="LegacyCRM"]
print(df_legacy)

                timestamp     source  \
60    2025-10-06 16:55:23  LegacyCRM   
255   2025-05-03 16:55:35  LegacyCRM   
377   2025-06-24 12:16:29  LegacyCRM   
1325  2025-04-17 07:33:44  LegacyCRM   
1734  2025-04-30 07:47:30  LegacyCRM   
1826  2025-01-23 10:33:36  LegacyCRM   
2217  2025-05-12 09:46:54  LegacyCRM   

                                            log_message         target_label  \
60    Lead conversion failed for prospect ID 7842 du...       Workflow Error   
255   API endpoint 'getCustomerDetails' is deprecate...  Deprecation Warning   
377   Customer follow-up process for lead ID 5621 fa...       Workflow Error   
1325  Escalation rule execution failed for ticket ID...       Workflow Error   
1734  The 'ExportToCSV' feature is outdated. Please ...  Deprecation Warning   
1826  Support for legacy authentication methods will...  Deprecation Warning   
2217  Task assignment for TeamID 3425 could not comp...       Workflow Error   

     complexity  cluster regex_label  

In [18]:
df_non_legacy = df_non_regex[df_non_regex.source!="LegacyCRM"]
print(df_non_legacy)

                timestamp           source  \
0     2025-06-27 07:20:25        ModernCRM   
1         1/14/2025 23:07        ModernCRM   
2          1/17/2025 1:29  AnalyticsEngine   
3     2025-07-12 00:24:16         ModernHR   
4     2025-06-02 18:25:23    BillingSystem   
...                   ...              ...   
2405  2025-08-13 07:29:25         ModernHR   
2406       1/11/2025 5:32         ModernHR   
2407  2025-08-03 03:07:47    ThirdPartyAPI   
2408     11/11/2025 11:52    BillingSystem   
2409     12/25/2025 13:21  AnalyticsEngine   

                                            log_message    target_label  \
0     nova.osapi_compute.wsgi.server [req-b9718cd8-f...     HTTP Status   
1        Email service experiencing issues with sending  Critical Error   
2             Unauthorized access to data was attempted  Security Alert   
3     nova.osapi_compute.wsgi.server [req-4895c258-b...     HTTP Status   
4     nova.osapi_compute.wsgi.server [req-ee8bc8ba-9...     HTTP Status 

In [19]:
print(df_non_legacy.shape)

(1903, 7)


In [20]:
model = SentenceTransformer('all-MiniLM-L6-v2')  # Lightweight embedding model
embeddings_filtered = model.encode(df_non_legacy['log_message'].tolist())

In [21]:
print(len(embeddings_filtered))

1903


In [22]:
X = embeddings_filtered
y = df_non_legacy['target_label'].values

In [23]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
clf = LogisticRegression(max_iter=1000)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
report = classification_report(y_test, y_pred)
print(report)

                precision    recall  f1-score   support

Critical Error       0.91      1.00      0.95        48
         Error       0.98      0.89      0.93        47
   HTTP Status       1.00      1.00      1.00       304
Resource Usage       1.00      1.00      1.00        49
Security Alert       1.00      0.99      1.00       123

      accuracy                           0.99       571
     macro avg       0.98      0.98      0.98       571
  weighted avg       0.99      0.99      0.99       571

